Yapılacaklar:
tek seferde sırayla tüm işlemleri yapacak kodu yaz

In [14]:
import pandas as pd
from collections import Counter

import os

import shutil
import torch
import torchvision

import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import classification_report

import copy

# import intel_extension_for_pytorch as ipex


# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt

In [2]:
input_choice = 1
input_choice = input("Seçim yapın:\n1. CPU için\n2. GPU (CUDA) için\n3. XPU (Intel ARC) için\n (Enter'a basmak CPU seçimi yapar)\n(1/2/3): ")
if input_choice == '1':
    device = torch.device("cpu")
elif input_choice == '2':
    device = torch.device("cuda")

elif input_choice == '3':
    device = torch.device("xpu") 


# device = torch.device("xpu" if torch.xpu.is_available() else "cpu")

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # grayscale için
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [4]:
data_dir = 'data_model/'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
    for x in ['train', 'validation', 'test']
}

class_names = image_datasets['train'].classes
print("Sınıflar:", class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']


## Model eğitecek fonksiyon

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=50, patience=5, class_names=None):
    """
    Derin öğrenme modelini eğitim ve doğrulama verisi üzerinde eğitir.

    Args:
        model (nn.Module): Eğitilecek PyTorch modeli.
        dataloaders (dict): 'train' ve 'validation' anahtarlarına sahip DataLoader sözlüğü.
        criterion (loss): Kayıp fonksiyonu.
        optimizer (torch.optim): Optimizasyon algoritması.
        device (torch.device): Eğitimde kullanılacak cihaz.
        num_epochs (int, optional): Maksimum eğitim dönemi sayısı. Varsayılan 50.
        patience (int, optional): Early stopping için tekrar süresi (doğruluk gelişmezse durma). Varsayılan 5.
        class_names (list, optional): Sınıf isimlerinin listesi. Sınıf bazlı doğruluk ve rapor için kullanılır. Varsayılan None.

    Returns:
        model (nn.Module): En iyi doğrulama başarımına sahip ağırlıklarla geri dönen model.
    """
    best_model_wts = model.state_dict()
    best_acc = 0.0
    epochs_no_improve = 0  # Early stopping için sayaç

    num_classes = len(class_names)


    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)


        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()   # Eğitim moduna geç
            else:
                model.eval()    # Değerlendirme moduna geç

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []


            # Her batch için döngü
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                if phase == 'validation':
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())


            # Epoch sonu istatistikleri
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # En iyi doğrulama modelini kaydet
            if phase == 'validation':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0  # İyileşme varsa sıfırla
                else:
                    epochs_no_improve += 1  # İyileşme yoksa artır
                print(f'{best_acc}, {epoch_acc}')


                # Sınıf bazlı başarı hesaplama
                if class_names:
                    class_correct = list(0. for i in range(num_classes))
                    class_total = list(0. for i in range(num_classes))
                    for i in range(len(all_labels)):
                        label = all_labels[i]
                        pred = all_preds[i]
                        if label == pred:
                            class_correct[label] += 1
                        class_total[label] += 1

                    print('\nSınıf Bazında Doğruluk:')
                    for i in range(num_classes):
                        if class_total[i] > 0:
                            accuracy = 100 * class_correct[i] / class_total[i]
                            print(f'Sınıf {i}: {accuracy:.2f}% ({int(class_correct[i])}/{int(class_total[i])})')
                        else:
                            print(f'Sınıf {i}: Hiç örnek bulunamadı.')


                    # Sınıf bazında rapor
                    print('\nSınıf Bazında Rapor:')
                    print(classification_report(all_labels, all_preds))

                else:
                    pass

        # Early stopping kontrolü
        if epochs_no_improve >= patience:
            print(f"\nErken durdurma tetiklendi. Doğrulama doğruluğu {patience} dönemi boyunca gelişmedi.")
            break

    # En iyi ağırlıkları geri yükle
    model.load_state_dict(best_model_wts)
    return model

## CNN modeli

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (1, 512, 512) -> (16, 256, 256)
        x = self.pool(F.relu(self.conv2(x)))  # (16, 256, 256) -> (32, 128, 128)
        x = self.pool(F.relu(self.conv3(x)))  # (32, 128, 128) -> (64, 64, 64)
        x = x.view(-1, 64 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## CustomCNN Model

In [7]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # -> (32, 224, 224)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (32, 112, 112)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (64, 56, 56)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (128, 28, 28)

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (256, 14, 14)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


## Modeli oluşturma

In [10]:
scratch_model = CustomCNN(num_classes=len(class_names)).to(device)

## Loss ve optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(scratch_model.parameters(), lr=0.001)

## Transfer Learning

In [ ]:
model = models.resnet18(pretrained=True)

# İlk katman RGB (3 kanal) bekliyor. 1 kanala uyarladık.
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Son katmanı sınıf sayısına göre değiştirdik.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Eğitime Başlatma

In [ ]:
j

In [ ]:
trained_model = train_model(model, dataloaders, criterion, optimizer, device)

In [12]:
trained_model = train_model(scratch_model, dataloaders, criterion, optimizer, device)


Epoch 1/50
--------------------
Train Loss: 2.8750 Acc: 0.1085
Validation Loss: 2.1975 Acc: 0.1111
0.1111111111111111, 0.1111111111111111

Epoch 2/50
--------------------
Train Loss: 2.1981 Acc: 0.1116
Validation Loss: 2.1984 Acc: 0.1133
0.11333333333333333, 0.11333333333333333

Epoch 3/50
--------------------
Train Loss: 2.1981 Acc: 0.1094
Validation Loss: 2.1973 Acc: 0.1111
0.11333333333333333, 0.1111111111111111

Epoch 4/50
--------------------
Train Loss: 2.1982 Acc: 0.1073
Validation Loss: 2.1972 Acc: 0.1111
0.11333333333333333, 0.1111111111111111

Epoch 5/50
--------------------
Train Loss: 2.1975 Acc: 0.1029
Validation Loss: 2.1972 Acc: 0.1111
0.11333333333333333, 0.1111111111111111

Epoch 6/50
--------------------
Train Loss: 2.1972 Acc: 0.1048
Validation Loss: 2.1977 Acc: 0.1111
0.11333333333333333, 0.1111111111111111

Epoch 7/50
--------------------
Train Loss: 2.1980 Acc: 0.1080
Validation Loss: 2.1978 Acc: 0.1111
0.11333333333333333, 0.1111111111111111

Erken durdurma teti

## Modeli kaydet

In [ ]:
model_save_path = "trained_model.pth"
torch.save(trained_model.state_dict(), model_save_path)
print(f"Model başarıyla kaydedildi: {model_save_path}")

## Modeli yniden oluştur

In [ ]:
model = SimpleCNN(num_classes=len(class_names))  # Model sınıfınızı yeniden tanımlayın
model.load_state_dict(torch.load("trained_model.pth"))
model = model.to(device)
model.eval()  # Modeli değerlendirme moduna alın
print("Model başarıyla yüklendi.")

## Modeli Kullanma

In [ ]:
with torch.no_grad():
    inputs = ...  # Yeni bir veri
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    print(f"Tahmin edilen sınıf: {class_names[preds[0]]}")